<a href="https://colab.research.google.com/github/OmriMan/EnsembleLearning_EnsembleDiversity_Assignment4/blob/main/Ensemble_Ass4_Stroke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages

In [114]:
import numpy as np  
import pandas as pd  

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from matplotlib import pyplot as plt  
from sklearn.tree import plot_tree 

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score

import random

from sklearn.model_selection import RandomizedSearchCV

from tabulate import tabulate

#Dataset Stroke Prediction#
source: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

Classify / predict whether a patient can suffer a stroke.

Dataset Attributes :

id : unique identifier

gender : "Male", "Female" or "Other"

age : age of the patient

hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension

heart_disease : 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease

ever_married : "No" or "Yes"

work_type : "children", "Govt_job", "Never_worked", "Private" or "Self-employed"

Residence_type : "Rural" or "Urban"

avg_glucose_level : average glucose level in blood

bmi : body mass index

smoking_status : "formerly smoked", "never smoked", "smokes" or "Unknown"*

stroke : 1 if the patient had a stroke or 0 if not

The data contains 5110 observations with 12 attributes.

source: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [115]:
def preprocess(df):
    # Gender
    change = { 'Other': 2,'Female':1,'Male':0}
    df.gender = df.gender.map(change)

    # BMI
    df['bmi'] = df['bmi'].replace(to_replace = np.nan, value =df['bmi'].mean())

    # ever_married
    change = {'No':0,'Yes':1}
    df.ever_married = df.ever_married.map(change)

    # Residence_type	
    change = {"Rural":1,"Urban":2}
    df.Residence_type	 = df.Residence_type.map(change)

    #work_type
    change = {"children":0, "Govt_job":1, "Never_worked":2, "Private":3 , "Self-employed":4}
    df.work_type = df.work_type.map(change)

    #smoking_status
    change = {"never smoked":0, "formerly smoked":1, "smokes":2, "Unknown":3}
    df.smoking_status	 = df.smoking_status.map(change)

    df = df.drop(columns=['id'])
    return df

Load the data set

In [116]:
df = pd.read_csv('/content/sample_data/stroke.csv')
df = preprocess(df)
features = ['gender',	'age',	'hypertension',	'heart_disease',	'ever_married',	'work_type',	'Residence_type',	'avg_glucose_level',	'bmi',	'smoking_status']
X = df.loc[:, features]
y = df.loc[:, ['stroke']]

Split the data into training and testin sets

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)
X_train

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
2954,1,18.0,0,0,0,3,2,70.54,23.5,3
1933,1,70.0,0,0,1,3,2,91.25,36.0,3
4311,1,65.0,0,0,1,3,1,205.78,41.7,0
2365,1,36.0,0,0,0,3,2,216.96,34.5,3
927,0,64.0,0,0,1,3,2,86.05,23.0,3
...,...,...,...,...,...,...,...,...,...,...
4931,1,53.0,1,0,1,1,1,98.61,38.8,2
3264,1,61.0,0,0,1,3,1,72.01,26.0,1
1653,0,44.0,0,0,1,3,1,94.71,28.4,2
2607,0,21.0,0,0,0,3,2,120.94,29.7,1


In [118]:
train = X_train

In [119]:
X_train

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
2954,1,18.0,0,0,0,3,2,70.54,23.5,3
1933,1,70.0,0,0,1,3,2,91.25,36.0,3
4311,1,65.0,0,0,1,3,1,205.78,41.7,0
2365,1,36.0,0,0,0,3,2,216.96,34.5,3
927,0,64.0,0,0,1,3,2,86.05,23.0,3
...,...,...,...,...,...,...,...,...,...,...
4931,1,53.0,1,0,1,1,1,98.61,38.8,2
3264,1,61.0,0,0,1,3,1,72.01,26.0,1
1653,0,44.0,0,0,1,3,1,94.71,28.4,2
2607,0,21.0,0,0,0,3,2,120.94,29.7,1


In [120]:
df = pd.read_csv('/content/sample_data/stroke.csv')
df.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

#Target class distribution#

In [121]:
target = df['stroke']
target.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

#Define and train the models #

Random Forest Classifier

In [122]:
model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train,np.ravel(y_train))

RandomForestClassifier()

In [123]:
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier = model_rf.predict(
    inputs
)
accuracy_RandomForestClassifier = accuracy_score(y_test,prediction_RandomForestClassifier)
print(f"Accuracy score of RandomForestClassifier = {accuracy_RandomForestClassifier}")

precision_RandomForestClassifier = precision_score(y_test,prediction_RandomForestClassifier,average='weighted')
print(f"Precision score of RandomForestClassifier = {precision_RandomForestClassifier}")

Accuracy score of RandomForestClassifier = 0.9475743348982786
Precision score of RandomForestClassifier = 0.9007897086454639


Extremely Randomized Trees

In [124]:
model_ExtraTreesClassifier = ExtraTreesClassifier(n_estimators=100)
model_ExtraTreesClassifier.fit(
    X_train,
    np.ravel(y_train)
)

ExtraTreesClassifier()

In [125]:
inputs = pd.DataFrame(X_test)
prediction_ExtraTreesClassifier = model_ExtraTreesClassifier.predict(
    inputs
)
accuracy_ExtraTreesClassifier = accuracy_score(y_test,prediction_ExtraTreesClassifier)
print(f"Accuracy score of ExtraTreesClassifier = {accuracy_ExtraTreesClassifier}")

precision_ExtraTreesClassifier = precision_score(y_test,prediction_ExtraTreesClassifier,average='weighted')
print(f"Precision score of ExtraTreesClassifier = {precision_ExtraTreesClassifier}")

Accuracy score of ExtraTreesClassifier = 0.9428794992175273
Precision score of ExtraTreesClassifier = 0.9103840172860932


Summary - Stroke dataset

In [126]:
print(tabulate([["Accuracy",accuracy_RandomForestClassifier,accuracy_ExtraTreesClassifier],["Precision",precision_RandomForestClassifier,precision_ExtraTreesClassifier]], headers=["Metric\Model","RandomForestClassifier","ExtraTreesClassifier"], tablefmt="grid"))

+----------------+--------------------------+------------------------+
| Metric\Model   |   RandomForestClassifier |   ExtraTreesClassifier |
+================+==========================+========================+
| Accuracy       |                 0.947574 |               0.942879 |
+----------------+--------------------------+------------------------+
| Precision      |                 0.90079  |               0.910384 |
+----------------+--------------------------+------------------------+


In [127]:
# i=0
# print(len(model_rf.estimators_))
# for t in model_rf.estimators_:
#   if i%2==0:
#     model_rf.estimators_.remove(t)
#   i+=0
# print("DHODI")
# print(len(model_rf.estimators_))

In [128]:
a = [5, 7, 11, 4, 5]
for i in range(1,len(a)):
  for previous, current in zip(a, a[i:]):
      print(previous, current)

5 7
7 11
11 4
4 5
5 11
7 4
11 5
5 4
7 5
5 5


In [129]:
def kohavi_wolpert(s,m):
  '''
  s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
  s[0] = x train
  s[1] = y train
  m - list of classifiers examined
  '''
  x = s[0]
  y=s[1]
  predictions =[]
  sum =0
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in m:
    preds = classifier.predict(x.values)
    predictions.append(preds)
  # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  l_i=0
  for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
    l_i=0
    for model_i_prediction in predictions:
      # l_i = number of classifiers misclassified x_i,y_i
      if model_i_prediction[pred_index] != y_i.values:
        l_i+=1
      #end sum all misclassification for x_i,y_i
    #sum = sum + l_i * (len(m) - l_i)
    sum = sum + ( l_i * ( len(m)-l_i ) ) 
  print(f"sum/(len(x)*(len(m)**2)) : {sum/(len(x)*(len(m)**2))}")
  return (sum) / ( len(y) * (len(m)**2) )



In [130]:
kohavi_wolpert((X_train,y_train),model_rf.estimators_)

sum/(len(x)*(len(m)**2)) : 0.02571260438413361


0.02571260438413361

In [131]:
def inter_rater_measure(s,m):
  '''
  s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
  s[0] = x train
  s[1] = y train
  m - list of classifiers examined
  '''
  x = s[0]
  y=s[1]
  predictions =[]
  sum =0
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in m:
    preds = classifier.predict(x.values)
    predictions.append(preds)
  # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  l=[]
  for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
    l_i=0
    for model_i_prediction in predictions:
      # l_i = number of classifiers misclassified x_i,y_i
      if model_i_prediction[pred_index] != y_i.values:
        l_i+=1
      #end sum all misclassification for x_i,y_i
    l.append(l_i)
    #sum = sum + l_i * (len(m) - l_i)
    sum = sum + ( l_i * ( len(m) - l_i) )
  # p = 1 - ( sum(l) / (|s| *|m|) )
  sum_l=0
  for val in l:
    sum_l+=val
  p = 1 - ( sum_l / (len(x)*len(m)) )
  result = 1 - (sum / ( len(x)*len(m)*(len(m)-1)*p*(1-p)  ) )
  return result

In [132]:
inter_rater_measure((X_train,y_train),model_rf.estimators_)

0.1586541795069698

In [133]:
import statistics
def deneral_diversity(s,m):
  '''
  s - the original training set tuple of (X_train,y_train) --> ((dataframe),((dataframe))
  s[0] = x train
  s[1] = y train
  m - list of classifiers examined
  '''
  x = s[0]
  y=s[1]
  predictions =[]
  # For each classifier, insert all its predictions (depending on the index of x) into the list .
  for classifier in m:
    preds = classifier.predict(x.values)
    predictions.append(preds)
  # Run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  v=[]
  for (index_x, x_i),(index_y, y_i),pred_index in zip(x.iterrows(),y.iterrows(),range(len(predictions[0]))):
    l_i=0
    for model_i_prediction in predictions:
      # l_i = number of classifiers misclassified x_i,y_i
      if model_i_prediction[pred_index] != y_i.values:
        l_i+=1
      #end sum all misclassification for x_i,y_i
    v_i= (len(m)-l_i) / len(m)
    v.append(v_i)
  # end of run through all the records in the training set and for each record : count the number of classifiers that made an incorrect classification(misclassified)
  # compute variance of v using statistics.variance
  variance_v = statistics.variance(v)
  return variance_v

In [134]:
deneral_diversity((X_train,y_train),model_rf.estimators_)

0.005158720659502825